**Step 1: Import packages**

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LogNorm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import tensorflow as tf

**Step 2: Define working directories**

In [2]:
os.chdir('C:/Users/user/Documents/Berkeley_MIDS/W207_Machine_Learning/w207_final_project')

**Step 3: Define classes**

**Step 4: Define functions**

**Step 5: Read data**

In [3]:
# Load npz files to combine and print content files
data_1 = np.load('et_w207_project_npz_files_5000_tmp_tmp5ocrhjnn.npz')
data_1.close

data_2 = np.load('et_w207_project_npz_files_5000_tmp_tmp092sag67.npz')
data_2.close

data_3 = np.load('et_w207_project_npz_files_5000_tmp_tmpf31_pn8p.npz')
data_3.close

data_4 = np.load('et_w207_project_npz_files_5000_tmp_tmpq5b2g4n2.npz')
data_4.close

data_key = np.concatenate((data_1['arr_0'], data_2['arr_0'], data_3['arr_0'], data_4['arr_0']))
data_pixel = np.concatenate((data_1['arr_1'], data_2['arr_1'], data_3['arr_1'], data_4['arr_1']))

df = pd.read_csv('school_project_data_set-query.csv')
# df = pd.read_csv('data/school_project_data_set-query.csv')


In [4]:
data_pixel

array([[-103.939    , -116.779    , -123.68     , ..., -103.939    ,
        -116.779    , -123.68     ],
       [ -46.939003 ,  -74.779    , -118.68     , ...,  -57.939003 ,
         -84.779    , -118.68     ],
       [ 151.061    ,  138.22101  ,  131.32     , ...,  151.061    ,
         138.22101  ,  131.32     ],
       ...,
       [  77.061    ,   46.221    ,   32.32     , ...,  -55.939003 ,
         -38.779    ,    1.3199997],
       [-102.939    , -115.779    , -122.68     , ...,  -68.939    ,
         -95.779    , -114.68     ],
       [ 112.061    ,  100.221    ,   89.32     , ...,   -5.939003 ,
          33.221    ,    3.3199997]], dtype=float32)

In [5]:
data_dict = {}
for i in range(data_key.shape[0]):
    data_dict[data_key[i]] = [data_pixel[i]]

df_pixel = pd.DataFrame(data=data_dict)
df_pixel_t = df_pixel.transpose()
df_pixel_t.index.names = ['attachment_key']
df_pixel_t.columns = ['pixel_array']
df_pixel_t

,pixel_array
attachment_key,
ozbpir-9ey6js-ggdqwo,"[-103.939, -116.779, -123.68, -103.939, -116.7..."
378ykanq,"[-46.939003, -74.779, -118.68, -24.939003, -57..."
p366md-big834-7o23k6,"[151.061, 138.22101, 131.32, 151.061, 138.2210..."
3zruwvl2,"[-3.939003, -66.779, -96.68, -3.939003, -66.77..."
q18iae-3vnh74-79npmy,"[-103.939, -116.779, -123.68, -103.939, -116.7..."
...,...
pktyig-481wyg-6gyulf,"[-91.939, -104.779, -97.68, -92.939, -105.779,..."
zu7y3rha,"[-1.939003, -14.778999, -21.68, -1.939003, -14..."
pxcwxf-8tnpyg-8vhb9r,"[77.061, 46.221, 32.32, 79.061, 49.221, 33.32,..."


In [6]:
merge_meta_pixel = df.merge(df_pixel_t, on='attachment_key')
combined_df = merge_meta_pixel[['attachment_key', 'size_mb', 'height', 'width', 'h_to_w',  'pixel_array', 'filename','logo']]
combined_df.head()

,attachment_key,size_mb,height,width,h_to_w,pixel_array,filename,logo
0,00xjny6u,0.011849,520.0,506.0,1.027668,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",Esterdale Theatre - Logo.png,1
1,04o31jop,0.012610,269.0,396.0,0.679293,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",BLUE_LOGO.png,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,"[27.060997, 14.221001, 13.32, 23.060997, 13.22...",ZEN_LB-5.jpg,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",article central 200.jpg,1
4,0bb8y85h,0.263813,885.0,800.0,1.106250,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",1116-100_WGTank_white.jpg,0


In [7]:
file_types = []
for item in combined_df['filename']:
    file_types.append(item[-3:].lower())

print(set(file_types))

{'pig', 'jpe', 'jpg', 'lue', '0mm', 'hot', 'peg', 'png', 'age', '73e', 'ack', 'ite'}


In [8]:
filetype_encoding = []
for item in combined_df['filename']:
    if item[-3:].lower() == 'jpg':
        filetype_encoding.append('jpg')
    elif item[-3:].lower() == 'png':
        filetype_encoding.append('png')
    else:
        filetype_encoding.append('other')


In [9]:
combined_df['filetype'] = filetype_encoding
combined_df = combined_df[['attachment_key', 'size_mb', 'height', 'width', 'h_to_w', 'filetype', 'pixel_array', 'filename','logo']]
combined_df.head()

C:\Users\yixzhang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,attachment_key,size_mb,height,width,h_to_w,filetype,pixel_array,filename,logo
0,00xjny6u,0.011849,520.0,506.0,1.027668,png,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",Esterdale Theatre - Logo.png,1
1,04o31jop,0.012610,269.0,396.0,0.679293,png,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",BLUE_LOGO.png,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,jpg,"[27.060997, 14.221001, 13.32, 23.060997, 13.22...",ZEN_LB-5.jpg,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,jpg,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",article central 200.jpg,1
4,0bb8y85h,0.263813,885.0,800.0,1.106250,jpg,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",1116-100_WGTank_white.jpg,0


In [10]:
# get dummies
cork_dummies = pd.get_dummies(combined_df.filetype)

# add to initial df
combined_df = pd.concat((combined_df, cork_dummies), axis=1)

# print head of df
combined_df.head()

,attachment_key,size_mb,height,width,h_to_w,filetype,pixel_array,filename,logo,jpg,other,png
0,00xjny6u,0.011849,520.0,506.0,1.027668,png,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",Esterdale Theatre - Logo.png,1,0,0,1
1,04o31jop,0.012610,269.0,396.0,0.679293,png,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",BLUE_LOGO.png,1,0,0,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,jpg,"[27.060997, 14.221001, 13.32, 23.060997, 13.22...",ZEN_LB-5.jpg,0,1,0,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,jpg,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",article central 200.jpg,1,1,0,0
4,0bb8y85h,0.263813,885.0,800.0,1.106250,jpg,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",1116-100_WGTank_white.jpg,0,1,0,0


**Step 6: Data preprocessing**

In [11]:
# Standardize pixel values between 0 and 1 by dividing by 255

In [12]:
combined_row_len, combined_column_len = combined_df.shape

In [13]:
# !!!!!! remember to random the data

In [14]:
# gray scale
gray_df = combined_df
gray_df.pixel_array = combined_df.pixel_array / 255.0
test_data = gray_df.iloc[:1000, :].values
train_data = gray_df.iloc[1000:combined_row_len, :].values

In [15]:
# gray scale
data_pixel_gray= data_pixel / 255.0
test_pixel_data = data_pixel_gray[:1000]
train_pixel_data = data_pixel_gray[1000:combined_row_len]

In [35]:
test_pixel_data.shape

(1000, 150528)

In [36]:
train_pixel_data.shape

(2755, 150528)

In [16]:
train_pixel_data

array([[ 0.5923961 ,  0.54204315,  0.51498044, ...,  0.5923961 ,
         0.54204315,  0.51498044],
       [ 0.43945488,  0.36165097,  0.23654902, ...,  0.52180785,
         0.4440039 ,  0.35811764],
       [-0.40760392, -0.45795685, -0.4850196 , ..., -0.40760392,
        -0.45795685, -0.4850196 ],
       ...,
       [ 0.23553333,  0.12243529,  0.04831373, ...,  0.34141567,
         0.22831765,  0.22870588],
       [ 0.5923961 ,  0.54204315,  0.51498044, ...,  0.5923961 ,
         0.54204315,  0.51498044],
       [-0.40760392, -0.45795685, -0.4850196 , ..., -0.3958392 ,
        -0.44619215, -0.4732549 ]], dtype=float32)

In [17]:
test_labels = gray_df.logo.values[:1000]
train_labels = gray_df.logo.values[1000:combined_row_len]

In [18]:
data_pixel_gray

array([[-0.40760392, -0.45795685, -0.4850196 , ..., -0.40760392,
        -0.45795685, -0.4850196 ],
       [-0.18407452, -0.29325098, -0.46541175, ..., -0.22721177,
        -0.33246666, -0.46541175],
       [ 0.5923961 ,  0.54204315,  0.51498044, ...,  0.5923961 ,
         0.54204315,  0.51498044],
       ...,
       [ 0.3022    ,  0.18125883,  0.12674509, ..., -0.21936864,
        -0.1520745 ,  0.00517647],
       [-0.40368235, -0.45403528, -0.48109803, ..., -0.27034903,
        -0.3756039 , -0.44972548],
       [ 0.43945488,  0.39302352,  0.3502745 , ..., -0.02329021,
         0.13027844,  0.01301961]], dtype=float32)

In [19]:
gray_df

,attachment_key,size_mb,height,width,h_to_w,filetype,pixel_array,filename,logo,jpg,other,png
0,00xjny6u,0.011849,520.0,506.0,1.027668,png,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",Esterdale Theatre - Logo.png,1,0,0,1
1,04o31jop,0.012610,269.0,396.0,0.679293,png,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",BLUE_LOGO.png,1,0,0,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,jpg,"[0.106121555, 0.05576863, 0.052235294, 0.09043...",ZEN_LB-5.jpg,0,1,0,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",article central 200.jpg,1,1,0,0
4,0bb8y85h,0.263813,885.0,800.0,1.106250,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",1116-100_WGTank_white.jpg,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3750,zu7y3rha,0.147371,498.0,702.0,0.709402,jpg,"[-0.007603933, -0.05795686, -0.08501961, -0.00...",journalism.jpg,0,1,0,0
3751,zuanbsqe,0.580392,206.0,415.0,0.496386,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",2013 05 26_techstars_id_final_bug solo graysca...,1,1,0,0
3752,zw9234v4,2.021135,1645.0,1450.0,1.134483,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",SPIN N GLO SILVER WING GPK MY.jpg,0,1,0,0
3753,zxfqbfj1,0.012057,36.0,345.0,0.104348,png,"[-0.40760392, -0.45795685, -0.4850196, -0.4076...",cornerstone web logo medium.png,1,0,0,1


In [37]:
def get_accuracy(num_pca):
    pca = PCA(n_components=num_pca)
    X_train_pca = pca.fit_transform(train_pixel_data)
    test_pca = pca.transform(test_pixel_data)

    # Create 2 GaussianMixture model,
    # one for the positive examples and one for the negative examples
    gm_logo = GaussianMixture(n_components=4, covariance_type='full').fit(X_train_pca[train_labels==1])
    gm_non_logo = GaussianMixture(n_components=4, covariance_type='full').fit(X_train_pca[train_labels==0])

    # Compare probability of the 2 GaussianMixture prediction
    pred = gm_logo.score_samples(test_pca) > gm_non_logo.score_samples(test_pca)
    print("Accuracy is", metrics.accuracy_score(test_labels, pred))

In [21]:
get_accuracy(2)

Accuracy is 0.489


In [22]:
get_accuracy(3)

Accuracy is 0.502


In [23]:
get_accuracy(4)

Accuracy is 0.515


In [24]:
get_accuracy(5)

Accuracy is 0.489


In [25]:
get_accuracy(10)

Accuracy is 0.492


In [26]:
get_accuracy(20)

Accuracy is 0.455


**Logistic Regression**

In [42]:
pca_logi = PCA(n_components=2)
X_train_pca = pca_logi.fit_transform(train_pixel_data)
test_pca = pca_logi.transform(test_pixel_data)
# lr = LogisticRegression(C=2.0, random_state=1, solver='lbfgs', multi_class='ovr')
lr = LogisticRegression(C=10.0)
lr.fit(X_train_pca, train_labels)
pred = lr.predict(test_pca)
print("Accuracy is", metrics.accuracy_score(test_labels, pred))

Accuracy is 0.704


C:\Users\yixzhang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Step 7: Single Layer NN**

7.1: Build the NN Model

7.2: Fit the NN Model

7.3: Evaluate performance on the test set

7.4: Make predictions

**Step 8: Multilayer NN**

8.1: Build the NN Model

8.2: Fit the NN Model

8.3: Evaluate performance on the test set

8.4: Make predictions

**Conclusions**